In [ ]:
import os, json
import pandas as pd
from tqdm.notebook import tqdm

with open(os.path.abspath('../data/TechQA/training_and_dev/training_Q_A.json')) as file:
    train_json = json.load(file)

with open(os.path.abspath('../data/TechQA/training_and_dev/dev_Q_A.json')) as file:
    validation_json = json.load(file)
    
technotes = pd.read_json("../data/TechQA/training_and_dev/training_dev_technotes.json", orient='index')

print('All files loaded...')

#Collecting all documents for a given record
def add_context(json_file):
    for row in tqdm(json_file):
        if row['ANSWERABLE'] == 'Y':
            row['context'] = technotes[technotes['id'] == row['DOCUMENT']].text[0]
        else:
            #Combine all documents to create a single document for a given unanswerable records.
            text = ''
            for doc_id in row['DOC_IDS']:
                text += technotes[technotes['id'] == doc_id].text[0]
            row['context'] = text

def remake_json(json_file):
    for idx, row in enumerate(json_file):
        row['document_id'] = idx
        row['question'] = row.pop('QUESTION_TEXT')
        row['is_impossible'] = True if row.pop('ANSWERABLE') == 'N' else False
        row['id'] = idx
        row['answers'] = {'text': [row.pop('ANSWER')], "answer_start": [row.pop('START_OFFSET')]} \
                         if row['is_impossible'] == False else \
                         {'text': [], 'answer_start':[]}
        for key in list(row):
            if key not in ['document_id', 'question', 'is_impossible', 'id', 'answers', 'context']:
                del row[key]

add_context(train_json)
add_context(validation_json)

remake_json(train_json)
remake_json(validation_json)

with open('training_Q_A_context.json', 'w+', encoding='utf-8') as output_file:
    for dic in train_json:
        json.dump(dic, output_file) 
        output_file.write("\n")

with open('dev_Q_A_context.json', 'w+', encoding='utf-8') as output_file:
    for dic in validation_json:
        json.dump(dic, output_file) 
        output_file.write("\n")

print('Files saved with context...')